# Post-processing for borefield histogram

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==3.5.1
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [ ]:
import cases as cas
import post_process_configurations as con


## Read annual results

This section also clears the old results to free up memory.

In [ ]:
# Free up storage, then read new data
import gc
gc.collect()
print("Cleaned up memory.")

Cleaned up memory.


In [ ]:
print("Reading result files")
r_base=con.get_results('base')


Reading result files
Read base results
Read base_hBor_0.8 results


#### Test generating the plots with Dymola simulation

# Heat pump size study: 

By gradually decreasing the central plant heat pump size, we violate the loop temperature requirement (within 10.5 degC to 24 degC) but reduce investments.

In [ ]:
LooTem = ["TLooMinMea.y", "TLooMaxMea.y", "datDis.TLooMin", "datDis.TLooMax", "datDis.QPlaHeaPumCoo_flow_nominal", "datDis.QPlaHeaPumHea_flow_nominal"]
heaPumSizCas = ['base', 'base_heaPumSizFac_0.9', 'base_heaPumSizFac_0.8']

In [ ]:
# read the partial results
resArr = []
for i in range(len(heaPumSizCas)):
    ele = heaPumSizCas[i]
    print("Reading ", ele, " result ----------")
    res = get_partial_results(ele, LooTem)
    res['name'] = ele
    resArr.append(res)
print("-------- Done! ----------")

In [ ]:
caseName = []
heaPumCap = []
maxLooTem = []
minLooTem = []
dTHou = []

for i in range(len(resArr)):
    ele = resArr[i]
    caseName.append(ele['name'])
    cooCap = ele['datDis.QPlaHeaPumCoo_flow_nominal'][0]
    heaCap = ele['datDis.QPlaHeaPumHea_flow_nominal'][0]
    heaPumCap.append(max(abs(cooCap), heaCap) / 1e6)   # converted to MW
    TLooMax = ele['datDis.TLooMax']
    TLooMin = ele['datDis.TLooMin']
    TLooMaxMea = ele['TLooMaxMea.y']
    TLooMinMea = ele['TLooMinMea.y']
    maxLooTem.append(max(TLooMaxMea) - 273.15)   # converted to degC
    minLooTem.append(min(TLooMinMea) - 273.15)   # converted to degC
    time = ele['time']
    dTHou.append(dT_hour(time, TLooMin, TLooMax, TLooMinMea, TLooMaxMea))

# central heat pump cost $1631/kW
uniCenHeaPumCos = 1631
# Note that now the heaPumCap has unit of MW
basCos = (uniCenHeaPumCos * heaPumCap[0])*1e3*1e-6
netCosCha = [(heaPumCap[i] - heaPumCap[0])*uniCenHeaPumCos*1e3*1e-6 for i in range(len(heaPumCap))]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(8, 6))

# print(dTHou)

width = 0.3
# Plot on the first subplot with two y-axes
ax1 = axes[0]
ax2 = ax1.twinx()
ax1.bar(caseName, heaPumCap, color='blue', label='Heat pump capacity', width=width*(-1), align='edge')
ax1.tick_params(axis='x', labelbottom=False)
ax1.set_ylabel('Heat pump capacity [MW]')
ax1.set_ylim(6,9)
ax1.tick_params(axis='y', colors='blue')
ax2.bar(caseName, netCosCha, color='red', label='Net cost change', width=width, align='edge')
ax2.set_ylabel('Investment cost change [million $]')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylim(0,4)
ax1.legend(loc='upper center', bbox_to_anchor=(0.2,1.0))
ax2.legend(loc='upper center', bbox_to_anchor=(0.6,1.0))

ax3 = axes[1]
ax4 = ax3.twinx()
ax3.bar(caseName, maxLooTem, color='blue', label='Maximum loop temperature', width=width, align='edge')
ax3.tick_params(axis='x', labelrotation=90)
ax3.set_ylabel(r'Temperature [$^\circ$C]')
ax3.set_ylim(20, 30)
ax3.tick_params(axis='y', colors='blue')
ax4.bar(caseName, dTHou, color='red', label='Degree hours', width=width, align='edge')
ax4.set_ylabel(r'Degree hours [$K \cdot h$]')
ax4.tick_params(axis='y', colors='red')
ax4.set_ylim(0,10)
ax3.legend(loc='upper center', bbox_to_anchor=(0.2,1.0))
ax4.legend(loc='upper center', bbox_to_anchor=(0.6,1.0))

# plt.title('Bar Plot with Two Y-Axes')
fig.tight_layout()
save_plot(plt, "central_heat_pump_sizes")